In [1]:
import numpy as np
import time
from flatland.envs.rail_generators import complex_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator
from flatland.envs.rail_env import RailEnv
from flatland.utils.rendertools import RenderTool
from flatland.envs.observations import GlobalObsForRailEnv
from flatland.envs.observations import TreeObsForRailEnv
from flatland.envs.rail_generators import complex_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_env import GridTransitionMap
# from utils.observation_utils import norm_obs_clip

In [77]:
NUMBER_OF_AGENTS = 10

nr_start_goal = 10
nr_extra = 2
min_dist = 5
max_dist = 99999
seed = np.random.randint(0,1000)

In [78]:
my_rail_generator = complex_rail_generator (
        nr_start_goal,
        nr_extra,
        min_dist,
        max_dist,
        seed
    )

env = RailEnv(
    
    # parameters taht control the dimensions of the map.
    width = 40,
    height = 40,
    
    # Rail generator: generates new rail networks on each reset
    # simplest rail generators: 
    #         envs.rail_generators.rail_from_manual_specifications_generator and 
    #         envs.rail_generators.random_rail_generator.
    
    rail_generator = my_rail_generator,
    schedule_generator = complex_schedule_generator(),
    obs_builder_object= GlobalObsForRailEnv(),
    number_of_agents = NUMBER_OF_AGENTS
)


In [79]:
# The original railway map from the complex_rail_generator
original_rail_map = env.rail.grid.copy()




In [82]:
# straightward implementation, could be optimized
# The node id is in the form (x,y), could be a linear mapping from 
# the most leftupper cornner to the most rightlower corner

# Flatland map encoding scheme, NESW -> N:[NESW], E:[NESW], S[NESW], W[NESW]
# Deadend cell is different, agent faces the opposite to the direction that it can goes. 


class decode_map:
    
    
    def __init__(self,original_map):
        self.original_map = original_map
        self.converted_input = {}

    # Determine next nodes for current_node.
    def case_matching(self, current_node, short_bits):
        
        next_nodes = []
        
        if (int(short_bits[0]) == 1):
            
            next_node = (current_node[0]-1, current_node[1])
            print("N", next_node)
            
            next_nodes.append(next_node)
        
        elif(int(short_bits[1]) == 1):
            
            next_node = (current_node[0], current_node[1]+1)
            print("E", next_node)
            
            next_nodes.append(next_node)

        
        elif(int(short_bits[2]) == 1):
            
            next_node = (current_node[0]+1, current_node[1])
            print("S", next_node)
            
            next_nodes.append(next_node)

        elif(int(short_bits[3]) == 1):
            
            next_node = (current_node[0], current_node[1]-1)
            print("W", next_node)
            
            next_nodes.append(next_node)
        
        return next_nodes
        
    # First find the possible previous nodes, and call another function to find the possible next nodes.
    def slice_16_bits(self, current_node, bits):
        
        previous_nodes = {}
        
        # Facing North
        
        if (bits[0:4] != "0000"):
            
            print(bits[0:4])
            next_nodes = self.case_matching(current_node, bits[0:4])
#             print("N, and returned next nodes", next_nodes)
            
            # add one previous node, and what nodes the agent can go from current node.
            previous_nodes[(current_node[0]+1,current_node[1])] = next_nodes
            
#             print("N, and current previous nodes", previous_nodes)
            
        # East
        if (bits[4:8] != "0000"):
            
            print(bits[4:8])
            next_nodes = self.case_matching(current_node, bits[4:8])
#             print("E",next_nodes)
            
            previous_nodes[(current_node[0], current_node[1]-1)] = next_nodes
            
#             print("E, and current previous nodes", previous_nodes)
# 

        # South
        if (bits[8:12] != "0000"):
            
            print(bits[8:12])
            next_nodes = self.case_matching(current_node, bits[8:12])
#             print("S",next_nodes)
            
            previous_nodes[(current_node[0]-1, current_node[1])] = next_nodes
            
#             print("S, and current previous nodes", previous_nodes)
       
        # West
        if (bits[12:16] != "0000"):
            
            print(bits[12:16])
            next_nodes = self.case_matching(current_node, bits[12:16])
#             print("W",next_nodes)
            
            previous_nodes[(current_node[0], current_node[1]+1)] = next_nodes
            
#             print("W, and current previous nodes", previous_nodes)

        return previous_nodes
            
    def convert_ori_rail_map(self):
        for row in range(0,len(self.original_map)):
            for col in range(0,len(self.original_map[row])):
                
                # Current node: (row,col)
                print("Current node:", (row,col))
                current_node = (row,col)
                # Get previous node, decided by which direction the agent is facing.
                # EXCEPT the deadend nodes.
                
                
#                 print('{0:016b}'.format(self.original_map[row][col]))
                previous_next_nodes = self.slice_16_bits(current_node, '{0:016b}'.format(self.original_map[row][col]))
                print(previous_next_nodes.items())
                
                for previous,direction in previous_next_nodes.items():
                    print("direction", direction)
                    if(len(direction)>1):
                        print("pair", previous, direction)
                    self.converted_input[(current_node, previous)] = direction
        print(self.converted_input)

In [83]:
decode_map(original_rail_map).convert_ori_rail_map()

Current node: (0, 0)
dict_items([])
Current node: (0, 1)
dict_items([])
Current node: (0, 2)
dict_items([])
Current node: (0, 3)
dict_items([])
Current node: (0, 4)
dict_items([])
Current node: (0, 5)
dict_items([])
Current node: (0, 6)
dict_items([])
Current node: (0, 7)
dict_items([])
Current node: (0, 8)
dict_items([])
Current node: (0, 9)
dict_items([])
Current node: (0, 10)
dict_items([])
Current node: (0, 11)
dict_items([])
Current node: (0, 12)
dict_items([])
Current node: (0, 13)
0100
E (0, 14)
dict_items([((0, 14), [(0, 14)])])
direction [(0, 14)]
Current node: (0, 14)
0001
W (0, 13)
0010
S (1, 14)
dict_items([((1, 14), [(0, 13)]), ((0, 13), [(1, 14)])])
direction [(0, 13)]
direction [(1, 14)]
Current node: (0, 15)
dict_items([])
Current node: (0, 16)
dict_items([])
Current node: (0, 17)
dict_items([])
Current node: (0, 18)
dict_items([])
Current node: (0, 19)
dict_items([])
Current node: (0, 20)
dict_items([])
Current node: (0, 21)
dict_items([])
Current node: (0, 22)
dict_it

1000
N (19, 31)
0010
S (21, 31)
dict_items([((21, 31), [(19, 31)]), ((19, 31), [(21, 31)])])
direction [(19, 31)]
direction [(21, 31)]
Current node: (20, 32)
dict_items([])
Current node: (20, 33)
dict_items([])
Current node: (20, 34)
dict_items([])
Current node: (20, 35)
dict_items([])
Current node: (20, 36)
dict_items([])
Current node: (20, 37)
dict_items([])
Current node: (20, 38)
dict_items([])
Current node: (20, 39)
dict_items([])
Current node: (21, 0)
dict_items([])
Current node: (21, 1)
dict_items([])
Current node: (21, 2)
dict_items([])
Current node: (21, 3)
1000
N (20, 3)
0010
S (22, 3)
dict_items([((22, 3), [(20, 3)]), ((20, 3), [(22, 3)])])
direction [(20, 3)]
direction [(22, 3)]
Current node: (21, 4)
dict_items([])
Current node: (21, 5)
dict_items([])
Current node: (21, 6)
dict_items([])
Current node: (21, 7)
dict_items([])
Current node: (21, 8)
dict_items([])
Current node: (21, 9)
dict_items([])
Current node: (21, 10)
1000
N (20, 10)
dict_items([((20, 10), [(20, 10)])])
dir

In [29]:
from flatland.core.grid.grid4 import Grid4Transitions
temp = Grid4Transitions(original_rail_map)
temp.get_direction_enum()

"<enum 'Grid4TransitionsEnum'>"

In [19]:
'{0:016b}'.format(33825)

'1000010000100001'

In [13]:
original_rail_map[6][5]

3

In [7]:
first_ob = env.reset()

In [8]:
print(len(first_ob[0]))

3


In [5]:
# tool to render environments
env_renderer = RenderTool(env, gl="PIL")

# env_renderer = RenderTool (env)
env_renderer.render_env(show=True)


C:\Users\yizh9\Anaconda3\envs\flatland-rl\lib\site-packages\flatland\utils\rendertools.py:289: UserWarning: Predictor did not provide any predicted cells to render.                 Observation builder needs to populate: env.dev_obs_dict
  Observation builder needs to populate: env.dev_obs_dict")
